In [1]:
# Import dependencies
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import json
import pandas as pd
from pandas.io import sql
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from consts import *
import time
%matplotlib inline

In [2]:
# Connecting to Postgres instance
engine = create_engine(CREATE_ENGINE_STR)

In [3]:
# Printing info for table names 
print (engine.table_names())

['agg_county_votes', 'committees', 'candidates', 'education', 'committee_summary_2020', 'agg_county_donors', 'six_state_donations', 'donations', 'fec_donor_az', 'health_metrics', 'birth_death_rate', 'postal_codes', 'fec_donor_mi', 'fec_donor_wi', 'fec_committee', 'fec_donor_pa', 'pres_votes_6t', 'unemployment', 'fec_donor_nc', 'fec_donor_fl']


In [4]:
def calculate_election_yr(i):
    return (2000 + i*4)

In [5]:
# Definition to take in the votes data frame. Function takes in the votes dataframe with 20 years of data. 
# This will loop thru every 4 years, runs thru all of the county votes then return it in a list. 
# This will aggregate everything and return a list in a df
def get_votes_intervals(votes_df, state_po):
    votes_states_df = votes_df[votes_df['state_po']==state_po]
    i = ELECTION_STARTING_YR
    
    four_yr_dfs = []
    while (i <= ELECTION_ENDING_YR):
        votes_states_interval_df = votes_states_df[votes_states_df['year']==i]    
        four_yr_dfs.append(votes_states_interval_df)
        i += ELECTION_INTERVAL
    
    return four_yr_dfs

In [6]:
# Goes thru each county (string), to pull the election date and calculate votes in the county that are democrat (blue), republic (red) and other. 
def vote_distribution(county, election_df, state, i):        
    county_df = election_df[election_df['county']==county]
    county_blue_df = county_df[county_df['party']==MAJOR_PARTIES[0]]
    county_red_df = county_df[county_df['party']==MAJOR_PARTIES[1]]  
    
    #Other = not democratic AND not republican  
    other_votes = 0
    blue_votes = pd.to_numeric(county_blue_df["candidatevotes"].sum(), errors='coerce')
    red_votes = pd.to_numeric(county_red_df["candidatevotes"].sum(), errors='coerce')
    
    unique_parties = county_df['party'].unique()
    for party in unique_parties:
        #Get a sum of all non major parties for other category
        if party not in MAJOR_PARTIES:
            party_df = county_df[county_df['party']==party]
            other_votes += pd.to_numeric(party_df["candidatevotes"].sum(), errors='coerce')
    
    #Total votes it the sum of blue + red + other
    total_votes = blue_votes + red_votes + other_votes
    
    election_year = calculate_election_yr(i)
    
    #Set the unemployment data points from the county for that election year.
    unemployment_sql = f'SELECT * FROM unemployment WHERE "County" = \'{county}\' AND "Stabr" = \'{state}\''
    unemployment_df = pd.read_sql_query(unemployment_sql,con=engine)
    
    winning_party = ''
    if blue_votes > red_votes:
        winning_party = MAJOR_PARTIES[0]
    elif red_votes > blue_votes:
        winning_party = MAJOR_PARTIES[1]
    else:
        winning_party = MAJOR_PARTIES[2]
        
    #Initialize 
    urban_pct = 0
    unemployment_rate = 0
    urban_den = 0
    rural_pct = 0
    rural_den = 0
    
    if not unemployment_df.empty:
        #Get POPPCT_URBAN -> urban_pct
        unemployment_col = "Unemployment_rate_" + str(election_year)  
        urban_pct = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), 'POPPCT_URBAN'].values[0]      
        unemployment_rate = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), unemployment_col].values[0]
        urban_den = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), 'POPDEN_URBAN'].values[0]
        rural_pct = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), 'POPPCT_RURAL'].values[0]
        rural_den = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), 'POPDEN_RURAL'].values[0]

    county_tuple = (
        blue_votes,
        red_votes,
        other_votes,
        total_votes,
        county,
        state,
        election_year,
        urban_pct,
        unemployment_rate,
        urban_den,
        rural_pct,
        rural_den,
        winning_party
    )
    return county_tuple

In [7]:
def donor_distribution(election_df, county, state, i):    
    county = county.strip()
    county_df = election_df[election_df['county']==county]
    
    #Convert the transaction_amt to a numeric column
    county_df["TRANSACTION_AMT"] = pd.to_numeric(county_df["TRANSACTION_AMT"], errors='coerce')
    
    county_blue_df = county_df[county_df['party']==MAJOR_PARTIES[0]]
    county_red_df = county_df[county_df['party']==MAJOR_PARTIES[1]]  
    
    #These vars hold the total amount donation per party
    other_amt = 0
    blue_amt = county_blue_df["TRANSACTION_AMT"].sum()
    red_amt = county_red_df["TRANSACTION_AMT"].sum()
    
    #These variables hold the number of donations per party
    other_num = 0
    blue_num = county_blue_df["TRANSACTION_AMT"].count()
    red_num = county_red_df["TRANSACTION_AMT"].count()
    
    unique_parties = county_df['party'].unique()
    for party in unique_parties:
        #Get a sum of all non major parties for other category
        if party not in MAJOR_PARTIES:
            party_df = county_df[county_df['party']==party]
            other_amt += party_df["TRANSACTION_AMT"].sum()
            other_num += party_df["TRANSACTION_AMT"].count()
    
    #Total transaction amount is the sum of blue + red + other
    total_amt = blue_amt + red_amt + other_amt
    #Total number of transactions is the sum of blue + red + other
    total_num = blue_num + red_num + other_num
        
    election_year = calculate_election_yr(i)
    donor_tuple = (
        blue_amt,
        red_amt,
        other_amt,
        total_amt,
        blue_num,
        red_num,
        other_num,
        total_num,
        county,
        state,
        election_year
    )
    return donor_tuple

In [8]:
# Run every election year in the loop, get election df, look for the vote distribution
# Will be run on a single state and will return a dictionary tha tcountains every county in the state as a key. 
# The value (number of votes) is a dictionary of values.
def county_vote_distribution(four_yr_dfs, state):
    #Loop through each election DF
    for i in range(len(four_yr_dfs)):
        election_df = four_yr_dfs[i]
        unique_counties = election_df["county"].unique()
        print(f"County election year: {calculate_election_yr(i)} num countines: {len(unique_counties)}")
        
        #Loop through each unique county
        for county in unique_counties:
            print(f"Votes for county: {county}")
            #Get the percent of the vote distribution for that county
            county_tuple = vote_distribution(county, election_df, state, i)            
            county_votes_df = pd.DataFrame([county_tuple], columns=VOTES_COLS)
            #Write the vote tallies per county to DB
            county_votes_df.to_sql(TABLE_AGG_VOTES, con=engine, if_exists="append")

In [9]:
"""
ACE	Ace Party	
AKI	Alaskan Independence Party	
AIC	American Independent Conservative	
AIP	American Independent Party	
AMP	American Party	
APF	American People's Freedom Party	
AE	Americans Elect	
CIT	Citizens' Party	
CMD	Commandments Party	
CMP	Commonwealth Party of the U.S.	
COM	Communist Party	
CNC	Concerned Citizens Party Of Connecticut	
CRV	Conservative Party	
CON	Constitution Party	
CST	Constitutional	
COU	Country	
DCG	D.C. Statehood Green Party	
DNL	Democratic -Nonpartisan League	
DEM	Democratic Party	
D/C	Democratic/Conservative	
DFL	Democratic-Farmer-Labor	
DGR	Desert Green Party	
FED	Federalist	
FLP	Freedom Labor Party	
FRE	Freedom Party	
GWP	George Wallace Party	
GRT	Grassroots	
GRE	Green Party	
GR	Green-Rainbow	
HRP	Human Rights Party	
IDP	Independence Party	
IND	Independent	
IAP	Independent American Party	
ICD	Independent Conservative Democratic	
IGR	Independent Green	
IP	Independent Party	
IDE	Independent Party of Delaware	
IGD	Industrial Government Party	
JCN	Jewish/Christian National	
JUS	Justice Party	
LRU	La Raza Unida	Also see RUP
LBR	Labor Party	Also see LAB
LFT	Less Federal Taxes	
LBL	Liberal Party	
LIB	Libertarian Party	
LBU	Liberty Union Party	
MTP	Mountain Party	
NDP	National Democratic Party	
NLP	Natural Law Party	
NA	New Alliance	
NJC	New Jersey Conservative Party	
NPP	New Progressive Party	
NPA	No Party Affiliation	
NOP	No Party Preference	Commonly used in CA & WA
NNE	None	
N	Nonpartisan	
NON	Non-Party	
OE	One Earth Party	
OTH	Other	
PG	Pacific Green	
PSL	Party for Socialism and Liberation	
PAF	Peace And Freedom	Also see PFP
PFP	Peace And Freedom Party	Also see PAF
PFD	Peace Freedom Party	
POP	People Over Politics	
PPY	People's Party	
PCH	Personal Choice Party	
PPD	Popular Democratic Party	
PRO	Progressive Party	
NAP	Prohibition Party	
PRI	Puerto Rican Independence Party	
RUP	Raza Unida Party	Also see LRU
REF	Reform Party	
REP	Republican Party	
RES	Resource Party	
RTL	Right To Life	
SEP	Socialist Equality Party	
SLP	Socialist Labor Party	
SUS	Socialist Party	
SOC	Socialist Party U.S.A.	
SWP	Socialist Workers Party	
TX	Taxpayers	
TWR	Taxpayers Without Representation	
TEA	Tea Party	
THD	Theo-Democratic	
LAB	U.S. Labor Party	Also see LBR
USP	U.S. People's Party	
UST	U.S. Taxpayers Party	
UN	Unaffiliated	
UC	United Citizen	
UNI	United Party	
UNK	Unknown	
VET	Veterans Party	
WTP	We the People	
W	Write-In
"""

"\nACE\tAce Party\t\nAKI\tAlaskan Independence Party\t\nAIC\tAmerican Independent Conservative\t\nAIP\tAmerican Independent Party\t\nAMP\tAmerican Party\t\nAPF\tAmerican People's Freedom Party\t\nAE\tAmericans Elect\t\nCIT\tCitizens' Party\t\nCMD\tCommandments Party\t\nCMP\tCommonwealth Party of the U.S.\t\nCOM\tCommunist Party\t\nCNC\tConcerned Citizens Party Of Connecticut\t\nCRV\tConservative Party\t\nCON\tConstitution Party\t\nCST\tConstitutional\t\nCOU\tCountry\t\nDCG\tD.C. Statehood Green Party\t\nDNL\tDemocratic -Nonpartisan League\t\nDEM\tDemocratic Party\t\nD/C\tDemocratic/Conservative\t\nDFL\tDemocratic-Farmer-Labor\t\nDGR\tDesert Green Party\t\nFED\tFederalist\t\nFLP\tFreedom Labor Party\t\nFRE\tFreedom Party\t\nGWP\tGeorge Wallace Party\t\nGRT\tGrassroots\t\nGRE\tGreen Party\t\nGR\tGreen-Rainbow\t\nHRP\tHuman Rights Party\t\nIDP\tIndependence Party\t\nIND\tIndependent\t\nIAP\tIndependent American Party\t\nICD\tIndependent Conservative Democratic\t\nIGR\tIndependent Green\t\

In [10]:
def map_zip_county(unique_zips, state_zips):
    county_dict = {}
    unique_counties = {}
    for zipcode_str in unique_zips:
        #If there is a problem casting the zip to an int, just skip it
        try:
            zipcode = int(zipcode_str)
        except ValueError as ve:
            print(ve)
            continue
            
        #Filter out on the zip code from the state_zips DF
        county_zip = state_zips[state_zips["zip"] == zipcode]
        #Get the county name from the DF and convert it to lower
        county_name = county_zip["county"].to_string(index=False).strip().lower()
        #Filter out the county string within
        county_name = county_name.replace(" county", "").capitalize()
        
        county_dict[zipcode_str] = county_name
        if county_name not in unique_counties:
            unique_counties[county_name] = True
            
    return (county_dict, unique_counties.keys())

In [11]:
def one_hot_encode(df):
    # Generate our categorical variable list
    cat_vars = df.dtypes[df.dtypes == "object"].index.tolist()

    # Create a OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)

    # Fit and transform the OneHotEncoder using the categorical variable list
    encode_df = pd.DataFrame(enc.fit_transform(df[cat_vars]))

    # Add the encoded variable names to the DataFrame
    encode_df.columns = enc.get_feature_names(cat_vars)
    
    return encode_df

In [12]:
#Add a new column party to the DF that maps the committee party abbreviation to a major party
def merge_cmtid_party(donor_df):        
    #Get the major party strings to map to 
    party_repub = MAJOR_PARTIES[1]
    party_democrat = MAJOR_PARTIES[0]
    party_other = "other"
    
    #Map the affiliation code to the party affiliation
    cmte_party_map = {
        "REP": party_repub,
        "TEA": party_repub,
        "DNL": party_democrat,
        "DNL": party_democrat,
        "DEM": party_democrat,
        "D/C": party_democrat,
        "DFL": party_democrat,
        "THD": party_democrat,
        "PPD": party_democrat,
        "UNK": party_other
    }
    
    donor_df["party"] = donor_df["cmte_pty_affiliation"].map(cmte_party_map)
    
    return donor_df

In [13]:
#Loop through each of the election year DFs and 
def donation_county_cycle_distribution(four_yr_dfs, state_zips, committee_df, state):
    #Loop through each election year DF
    for i in range(len(four_yr_dfs)):
        election_df = four_yr_dfs[i]
        #Without zipcode can't do a county lookup, so drop all null values
        election_df.dropna(subset=["ZIP"], inplace=True)
        #Get the unique values of zip code in the election DF
        unique_zips = election_df["ZIP"].unique()
        #Createa a map of zip to county, and a list of all unique counties in that state
        (zip_county_map, unique_counties) = map_zip_county(unique_zips, state_zips)
        #Map the zipcode to the county name per the map function
        election_df["county"] = election_df["ZIP"].map(zip_county_map)
        
        print(f"Donor distribution election yr: {calculate_election_yr(i)} num counties: {len(unique_counties)}")

        #Loop through each unique county
        for county in unique_counties:
            print(f"Donations for county: {county}")
            #Get the donor distribution for that county, state, election year as a tuple
            donor_tuple = donor_distribution(election_df, county, state, i)
            #Create a DF to store the county donor info
            donor_df = pd.DataFrame([donor_tuple], columns = DONOR_COLS)  
            #Write the donation amounts to the DB
            donor_df.to_sql(TABLE_AGG_DONORS, con=engine, if_exists="append")

In [14]:
def str_dt(donor_date_str):
    #01/01/1996 - 12/31/1999
    donor_date = datetime.strptime(donor_date_str, '%m%d%Y')
    return donor_date

In [15]:
def get_year_from_date_str(donor_date_str):
    donor_date = str_dt(donor_date_str)
    donor_year = donor_date.year
    return donor_year

In [16]:
def get_donors_intervals(donors_states_df):
    donors_states_df['trans_date'] =  pd.to_datetime(donors_states_df['TRANSACTION_DT'], format='%m%d%Y', errors="coerce")
    
    i = ELECTION_STARTING_YR
    prev_year = ELECTION_STARTING_YR - ELECTION_INTERVAL
    
    four_yr_dfs = []
        
    while (i <= ELECTION_ENDING_YR):
        votes_states_interval_df = donors_states_df[(donors_states_df['trans_date'].dt.year > prev_year) & (donors_states_df['trans_date'].dt.year <= i)]          
        four_yr_dfs.append(votes_states_interval_df)
        i += ELECTION_INTERVAL
        prev_year += ELECTION_INTERVAL
        
    return four_yr_dfs

In [17]:
#Get all donation records for a single state and return it in a dataframe
def donor_state_query(state, engine):
    #Run queries to get all donation records from the states into dfs
    donor_table_name = '"fec_donor_{}"'.format(state.lower())    
    donor_select_sql = 'select * from {}'.format(donor_table_name)
    donor_df = pd.read_sql_query(donor_select_sql,con=engine)
    return donor_df

In [18]:
#Get all donation records for a single state and return it in a dataframe
def donor_six_state_query(state):
    donor_select_sql = f'select * from six_state_donations where "STATE" = \'{state}\''
    donor_df = pd.read_sql_query(donor_select_sql,con=engine)
    return donor_df

In [19]:
#Aggregate tables are the output of this script, drop them to start fresh
def drop_agg_tables():
    sql.execute('DROP TABLE IF EXISTS %s'%TABLE_AGG_DONORS, engine)
    sql.execute('DROP TABLE IF EXISTS %s'%TABLE_AGG_VOTES, engine)

In [20]:
#Main Loop of the program
def main():
    print("Main")
    start_main = time.time()
    
    #Read the various tables into DFs
    health_df = pd.read_sql_query('select * from "health_metrics"',con=engine)
    committee_df = pd.read_sql_query('select * from "fec_committee"',con=engine)
    votes_df = pd.read_sql_query('select * from "pres_votes_6t"',con=engine)
    zips_df = pd.read_sql_query('select * from "postal_codes"',con=engine)
    
    #Lowercase the column
    committee_df['cmte_id'] = committee_df['cmte_id'].str.upper()
    
    #Drop the aggregate tables to do fresh data analysis
    drop_agg_tables()
    
    #Loop through each state
    for state in SWING_STATES:
        print("State:" + state)
        print("Aggregating Vote and Donation records...")
        #Get the votes related to that state
        votes_intervals_df = get_votes_intervals(votes_df, state)

        #Get the distribution of Red, Blue, and Other votes in a list of dict per election yr e.g. 2000 + 4n
        county_vote_distribution(votes_intervals_df, state)
        
        print("Doing Donor Query")
        #DF that has all donation records for a state
        start_q = time.time()
        donor_df_state = donor_six_state_query(state)
        end_q = time.time()
        time_q = end_q - start_q
        print(f"Done Donor Query time: {time_q}")
        
        #Add party column to donor data frame
        donor_df = committee_df.merge(donor_df_state, left_on='cmte_id', right_on='CMTE_ID')
        donor_df = merge_cmtid_party(donor_df)
        
        #Get a list of DFs per election year per state
        donors_intervals_df = get_donors_intervals(donor_df)
        #Filter out the zips DF by the state
        state_zips = zips_df[zips_df["state"] == state]
                
        #Get list of tuples 
        donation_county_cycle_distribution(donors_intervals_df, state_zips, committee_df, state)
        
        print("\n")
    
    print("Swing States Aggregation Done!")
    end_main = time.time()
    time_main = end_main - start_main
    print(time_main)

In [21]:
#Run the main loop
main()

Main
State:AZ
Aggregating Vote and Donation records...
County election year: 2000 num countines: 15
Votes for county: Apache
Votes for county: Cochise
Votes for county: Coconino
Votes for county: Gila
Votes for county: Graham
Votes for county: Greenlee
Votes for county: La Paz
Votes for county: Maricopa
Votes for county: Mohave
Votes for county: Navajo
Votes for county: Pima
Votes for county: Pinal
Votes for county: Santa Cruz
Votes for county: Yavapai
Votes for county: Yuma
County election year: 2004 num countines: 15
Votes for county: Apache
Votes for county: Cochise
Votes for county: Coconino
Votes for county: Gila
Votes for county: Graham
Votes for county: Greenlee
Votes for county: La Paz
Votes for county: Maricopa
Votes for county: Mohave
Votes for county: Navajo
Votes for county: Pima
Votes for county: Pinal
Votes for county: Santa Cruz
Votes for county: Yavapai
Votes for county: Yuma
County election year: 2008 num countines: 15
Votes for county: Apache
Votes for county: Cochise

/Users/C454479/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/C454479/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


invalid literal for int() with base 10: '8501P'
Donor distribution election yr: 2000 num counties: 27
Donations for county: Mohave


/Users/C454479/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Donations for county: Maricopa
Donations for county: Pima
Donations for county: Yavapai
Donations for county: Cochise
Donations for county: Coconino
Donations for county: Apache
Donations for county: Santa cruz
Donations for county: Series([], )
Donations for county: Coconino
  yavapai
Donations for county: Yuma
Donations for county: Cochise
  graham
Donations for county: Graham
Donations for county: La paz
Donations for county: Cochise
 santa cruz
Donations for county: Gila
Donations for county: Maricopa
    pinal
Donations for county: Navajo
Donations for county: Apache
 navajo
Donations for county: Cochise
       pima
 santa cruz
Donations for county: Maricopa
  yavapai
Donations for county: Pinal
Donations for county: Pima
 santa cruz
Donations for county: Greenlee
Donations for county: Coconino
   navajo
Donations for county: Pima
 pinal
Donations for county: Gila
 pinal
Donor distribution election yr: 2004 num counties: 28
Donations for county: Pima
Donations for county: Maricopa

...

Donations for county: Marinette
Donations for county: Columbia
     dane
Donations for county: Bayfield
Donations for county: Winnebago
Donations for county: Chippewa
 eau claire
Donations for county: Brown
     door
 kewaunee
 marathon
Donations for county: Florence
 marinette
Donations for county: Buffalo
       dunn
 eau claire
      pepin
Donations for county: Pierce
 st. croix
Donations for county: Pierce
Donations for county: Trempealeau
Donations for county: Adams
 portage
    wood
Donations for county: Jefferson
      rock
  walworth
Donations for county: Eau claire
Donations for county: Portage
  waupaca
 waushara
Donations for county: Dodge
 washington
Donations for county: Richland
     sauk
Donations for county: Iowa
Donations for county: Fond du lac
   winnebago
Donations for county: Iowa
 richland
     sauk
Donations for county: Outagamie
 winnebago
Donations for county: Dodge
 fond du lac
  washington
Donations for county: Grant
 lafayette
Donations for county: Iron
Dona

Donations for county: La crosse
Donations for county: Washington
   waukesha
Donations for county: Ozaukee
 washington
   waukesha
Donations for county: Rock
Donations for county: Iowa
Donations for county: Door
Donations for county: Marinette
    oconto
Donations for county: Walworth
Donations for county: Adams
Donations for county: Iron
Donations for county: Kenosha
   racine
 walworth
Donations for county: Green
Donations for county: Portage
Donations for county: Columbia
     sauk
Donations for county: Columbia
      dodge
 green lake
Donations for county: Marquette
  waushara
Donations for county: Fond du lac
Donations for county: Outagamie
Donations for county: Richland
Donations for county: Washburn
Donations for county: Sheboygan
Donations for county: Chippewa
Donations for county: Waupaca
Donations for county: Burnett
    polk
Donations for county: Bayfield
   sawyer
Donations for county: La crosse
    monroe
    vernon
Donations for county: Jefferson
  waukesha
Donations for 

Donations for county: Rock
 walworth
Donations for county: Kenosha
 walworth
Donations for county: Green lake
   waushara
  winnebago
Donations for county: Adams
 marquette
  waushara
Donations for county: Calumet
 fond du lac
Donations for county: Oconto
 shawano
Donations for county: Adams
  portage
 waushara
Donations for county: Eau claire
     jackson
 trempealeau
Donations for county: Barron
 washburn
Donations for county: Lincoln
Donations for county: Clark
  wood
Donations for county: Barron
      dunn
    pierce
      polk
 st. croix
Donations for county: Marathon
  portage
     wood
Donations for county: Buffalo
    dunn
   pepin
Donations for county: Langlade
Donations for county: Menominee
    oconto
   shawano
Donations for county: Grant
 lafayette
Donations for county: Dunn
Donations for county: Pepin
 pierce
Donations for county: Portage
 waupaca
Donations for county: Waupaca
 waushara
Donations for county: Columbia
 green lake
Donations for county: Juneau
 monroe
 verno

Donations for county: Juneau
Donations for county: Iowa
 lafayette
Donations for county: Clark
  wood
Donations for county: Wood
Donations for county: Jackson
 trempealeau
Donations for county: Jackson
  juneau
  monroe
Donations for county: Clark
 jackson
    wood
Donations for county: Burnett
 washburn
Donations for county: Barron
Donations for county: Adams
 portage
    wood
Donations for county: Burnett
Donations for county: Oconto
 shawano
Donations for county: Clark
 taylor
Donations for county: Fond du lac
   sheboygan
Donations for county: Columbia
    dodge
Donations for county: Dodge
 fond du lac
  washington
Donations for county: Ozaukee
 sheboygan
Donations for county: Fond du lac
  green lake
   winnebago
Donations for county: Buffalo
    dunn
   pepin
Donations for county: Marquette
  waushara
Donations for county: Green lake
Donations for county: Buffalo
Donations for county: Dunn
Donations for county: Brown
  kewaunee
 manitowoc
Donations for county: Pepin
Donations for

Donations for county: Langlade
Donations for county: Grant
      iowa
 lafayette
Donations for county: Brown
 outagamie
   shawano
Donations for county: Green lake
  marquette
   waushara
Donations for county: Eau claire
Donations for county: Dodge
 fond du lac
  washington
Donations for county: Marinette
    oconto
Donations for county: Columbia
     dane
    dodge
Donations for county: Dane
 iowa
Donations for county: Green lake
  marquette
Donations for county: Dodge
 washington
Donations for county: Forest
 oneida
Donations for county: Milwaukee
    racine
Donations for county: Brown
     door
 kewaunee
 marathon
Donations for county: Buffalo
    dunn
   pepin
Donations for county: Calumet
 winnebago
Donations for county: Calumet
 manitowoc
 sheboygan
Donations for county: Brown
   calumet
 outagamie
Donations for county: Outagamie
   waupaca
Donations for county: Portage
 waupaca
Donations for county: Fond du lac
   winnebago
Donations for county: Brown
   calumet
 manitowoc
Donat